In [1]:
from langchain_community.llms import Ollama

In [2]:
llm = Ollama(model="phi3")
llm.invoke("The first man on the moon was ...")

'Neil Armstrong. He became the first person to step onto the lunar surface on July 20, 1969, during NASA\'s Apollo 11 mission. His famous quote upon stepping onto the Moon is, "That\'s one small step for [a] man, one giant leap for mankind."'

In [3]:
llm.invoke("Can you add an appropriate category next to each of the following expenses.Respond with a list of categories such that Lidl Dig Trading GmbHCo as Groceries,Bäcker Dries sagt Danke as Restaurants,PH Pizza Hut FRANKFURT as Restaurants,TEDi Fil Mainz KFN as Shopping..According to above things do categorize below Ramesh Prasadam Scalable Capital Broker saving, PayPal Europe S.a.r.l. et Cie S.C.A,Bargeldauszahlung Deutsche Bank///DE,'/MISTRAL_SO//ZZ1FWEQ15VWLKMJTH//CHA N//SFA//USTRD//Kartenzahlung ARAL Mainz-Kostheim Hochheimer Stra")

'1. Ramesh Prasadam Scalable Capital Broker saving - Investments/Finance\n2. PayPal Europe S.a.r.l. et Cie S.C.A - Electronic Transactions/Payment Services\n3. Bargeldauszahlung Deutsche Bank///DE - Banking/Money Transfer\n4. Kartenzahlung ARAL Mainz-Kostheim Hochheimer Stra - Retail/Local Payments'

In [4]:
import pandas as pd
import datetime
import re
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns


In [5]:
#Read csv File
df =pd.read_csv("ardata.csv")

In [6]:
df

,Date,Name / Description,Amount (EUR)
0,2024-01-02,Income from Salary,1150.00
1,2024-01-02,Dinner at Hindukus Restaurant,-40.80
2,2024-01-02,Grocery shopping at Desi Grocery,-20.00
3,2024-01-02,Grocery shopping at Lidl,-8.28
4,2024-01-02,Grocery shopping at Supermarket Istanbul,-25.34
...,...,...,...
112,2024-05-21,SUPERMARKET ISTANBUL,-18.10
113,2024-05-21,DM MOVIE,-21.90
114,2024-05-22,Cash Withdrawal. Debit Card,-100.00
115,2024-05-22,Cash Deposit SB SB-DEPOSIT HARRAS,870.00


In [7]:
#Adding a column Income to know Income and Expenses
df['income/expenses']= np.where(df['Amount (EUR)'] >= 0, 'income', 'expenses')

In [8]:
df


,Date,Name / Description,Amount (EUR),income/expenses
0,2024-01-02,Income from Salary,1150.00,income
1,2024-01-02,Dinner at Hindukus Restaurant,-40.80,expenses
2,2024-01-02,Grocery shopping at Desi Grocery,-20.00,expenses
3,2024-01-02,Grocery shopping at Lidl,-8.28,expenses
4,2024-01-02,Grocery shopping at Supermarket Istanbul,-25.34,expenses
...,...,...,...,...
112,2024-05-21,SUPERMARKET ISTANBUL,-18.10,expenses
113,2024-05-21,DM MOVIE,-21.90,expenses
114,2024-05-22,Cash Withdrawal. Debit Card,-100.00,expenses
115,2024-05-22,Cash Deposit SB SB-DEPOSIT HARRAS,870.00,income


In [9]:
unique_transactions = df["Name / Description"].unique()
len(unique_transactions)

50

In [10]:
unique_transactions[0:60]

array(['Income from Salary', 'Dinner at Hindukus Restaurant',
       'Grocery shopping at Desi Grocery', 'Grocery shopping at Lidl',
       'Grocery shopping at Supermarket Istanbul', 'Cash Withdrawal',
       'Personal Care Products Purchase', 'Utility Bill Payment',
       'Miscellaneous Expense', 'DB payment', 'Fee for Service',
       'Gym Membership Fee', 'Insurance Premium Payment',
       'Grocery shopping at Paryan Lebensmittel',
       'Grocery shopping at Orient Asia Supermarket',
       'Dinner at Palmen Grill', 'Uncategorised Transaction',
       'Grocery shopping at Haffez Bazar', 'Dinner at Sara-H Restaurant',
       'Clothing shopping at Mode Paris', 'Fees for Bank Account',
       'Income from Cashback', 'Clothing shopping at SHEIN.COM',
       'Payment at Lidl', 'Payment for Services', 'Statement',
       'Credit Transfer', 'RSG Group GmbH M--- M- McFIT: Summer',
       'RESTAURANT NIGIN', 'DB Sales Company Subscription to',
       'Telefonica Germany GmbH + Co', 'LIDL

In [11]:
def hop(start, stop, step):
    for i in range(start, stop, step):
        yield i
    yield stop

index_list = list(hop(0, len(unique_transactions), 30))
index_list

[0, 30, 50]

In [12]:
def categorize_transactions(transaction_names, llm):
    response = llm.invoke("Can you add an appropriate category to the following expenses. For example: Dinner at Hindukus Restaurant - Food, Grocery shopping at Desi Grocery - Grocery,Income from Salary -Income  etc. Grocery shopping at Lidl - Grocery. " + transaction_names)
    response = response.split('\n')

    print(response)

    # Put in dataframe
    categories_df = pd.DataFrame({'Transaction vs category': response})
    categories_df[['Transaction', 'Category']] = categories_df['Transaction vs category'].str.split(' - ', expand=True)

    return categories_df

In [13]:
categorize_transactions('transaction_names',
                        llm)

['Grocery Shopping at Lidl - Grocery/Food Expenses', '', 'This category is appropriate because the expense relates directly to purchasing food items and household supplies, which are classified under groceries. Additionally, if the purchased items were used for cooking or consumption, it could also fall under Food Expenses.']


,Transaction vs category,Transaction,Category
0,Grocery Shopping at Lidl - Grocery/Food Expenses,Grocery Shopping at Lidl,Grocery/Food Expenses
1,,,None
2,This category is appropriate because the expen...,This category is appropriate because the expen...,None


In [19]:
# Intialise the categories_df_all dataframe
categories_df_all = pd.DataFrame()

# Loop through the index_list
for i in range(0, len(index_list)-1):
    transaction_names = unique_transactions[index_list[i]:index_list[i+1]]
    transaction_names = ','.join(transaction_names)

    categories_df = categorize_transactions(transaction_names, llm)
    categories_df_all = pd.concat([categories_df_all, categories_df], ignore_index=True)

['1. Dinner at Hindukus Restaurant - Food & Entertainment', '2. Grocery shopping at Desi Grocery - Groceries', '3. Income from Salary - Income', '4. Dinner at Hindukus Restaurant - Food & Entertainment', '5. Grocery shopping at Lidl - Groceries', '6. Cash Withdrawal - Financial Transactions', '7. Personal Care Products Purchase - Household Items', '8. Utility Bill Payment - Bills and Subscriptions', '9. Miscellaneous Expense - Other', '10. DB payment - Business Expenses', '11. Fee for Service - Services', '12. Gym Membership Fee - Health & Wellness', '13. Insurance Premium Payment - Insurance', '14. Grocery shopping at Paryan Lebensmittel - Groceries', '15. Dinner at Palmen Grill - Food & Entertainment', '16. Uncategorised Transaction - Miscellaneous', '17. Grocery shopping at Haffez Bazar - Groceries', '18. Dinner at Sara-H Restaurant - Food & Entertainment', '19. Clothing shopping at Mode Paris - Apparel and Accessories', '20. Fees for Bank Account - Financial Transactions', '21. Inc

In [20]:
categories_df_all

,Transaction vs category,Transaction,Category
0,1. Dinner at Hindukus Restaurant - Food & Ente...,1. Dinner at Hindukus Restaurant,Food & Entertainment
1,2. Grocery shopping at Desi Grocery - Groceries,2. Grocery shopping at Desi Grocery,Groceries
2,3. Income from Salary - Income,3. Income from Salary,Income
3,4. Dinner at Hindukus Restaurant - Food & Ente...,4. Dinner at Hindukus Restaurant,Food & Entertainment
4,5. Grocery shopping at Lidl - Groceries,5. Grocery shopping at Lidl,Groceries
5,6. Cash Withdrawal - Financial Transactions,6. Cash Withdrawal,Financial Transactions
6,7. Personal Care Products Purchase - Household...,7. Personal Care Products Purchase,Household Items
7,8. Utility Bill Payment - Bills and Subscriptions,8. Utility Bill Payment,Bills and Subscriptions
8,9. Miscellaneous Expense - Other,9. Miscellaneous Expense,Other
9,10. DB payment - Business Expenses,10. DB payment,Business Expenses


In [21]:
 categories_df_all.to_csv("categories_df_all.csv", index=False)

In [22]:
# Get unique categories in categories_df_all
unique_categories = categories_df_all["Category"].unique()
unique_categories

array(['Food & Entertainment', 'Groceries', 'Income',
       'Financial Transactions', 'Household Items',
       'Bills and Subscriptions', 'Other', 'Business Expenses',
       'Services', 'Health & Wellness', 'Insurance', 'Miscellaneous',
       'Apparel and Accessories', 'Groceries/Food', '**Grocery**',
       '**Entertainment/Subscriptions**',
       '**Dining & Grocery Shopping** (Grocery if only the shopping part is considered)',
       '**Online Purchases**',
       '**Investments & Subscriptions** (Considering the investment aspect of Citizen Bonus)',
       '**Transactions/Withdrawals**',
       '**Subscriptions & Transactions/Withdrawals**', '**Deposits**'],
      dtype=object)

In [23]:
# Drop NA values
categories_df_all = categories_df_all.dropna()
# If category contains "Food", then categorise as "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Food"), 'Category'] = "Food and Drinks"
# If category contains "Clothing", then categorise as "Clothing"
categories_df_all.loc[categories_df_all['Category'].str.contains("Clothing"), 'Category'] = "Clothing"
# If category contains "Services", then categorise as "Services"
categories_df_all.loc[categories_df_all['Category'].str.contains("Services"), 'Category'] = "Services"
# If category contains "Health" or "Wellness", then categorise as "Health and Wellness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Health|Wellness"), 'Category'] = "Health and Wellness"
# If category contains "Sport", then categorise as "Sport
#  and Fitness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Sport"), 'Category'] = "Sport and Fitness"
# If category contains "Travel", then categorise as "Travel"
categories_df_all.loc[categories_df_all['Category'].str.contains("Travel"), 'Category'] = "Travel"

In [ ]:
categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'^\d+[\.\₁₂₃₄₅₆₇₈₉]\s+', '',regex = True)
categories_df_all

In [24]:
# Remove the numbering eg "1. " from Transaction column
categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '',regex = True)
categories_df_all


,Transaction vs category,Transaction,Category
0,1. Dinner at Hindukus Restaurant - Food & Ente...,Dinner at Hindukus Restaurant,Food and Drinks
1,2. Grocery shopping at Desi Grocery - Groceries,Grocery shopping at Desi Grocery,Groceries
2,3. Income from Salary - Income,Income from Salary,Income
3,4. Dinner at Hindukus Restaurant - Food & Ente...,Dinner at Hindukus Restaurant,Food and Drinks
4,5. Grocery shopping at Lidl - Groceries,Grocery shopping at Lidl,Groceries
5,6. Cash Withdrawal - Financial Transactions,Cash Withdrawal,Financial Transactions
6,7. Personal Care Products Purchase - Household...,Personal Care Products Purchase,Household Items
7,8. Utility Bill Payment - Bills and Subscriptions,Utility Bill Payment,Bills and Subscriptions
8,9. Miscellaneous Expense - Other,Miscellaneous Expense,Other
9,10. DB payment - Business Expenses,DB payment,Business Expenses


In [30]:
# Merge the categories_df_all with the transactions_2022_2023.csv dataframe (df)
df = pd.read_csv("ardata.csv")
df.loc[df['Name / Description'].str.contains("Hindukus"), 'Name / Description'] = "Dinner at Hindukus Restaurant"
df = pd.merge(df, categories_df_all, left_on='Name / Description', right_on ='Transaction',how = 'right')
df.head(80)

,Date,Name / Description,Amount (EUR),Transaction vs category,Transaction,Category
0,2024-01-02,Dinner at Hindukus Restaurant,-40.80,1. Dinner at Hindukus Restaurant - Food & Ente...,Dinner at Hindukus Restaurant,Food and Drinks
1,2024-01-02,Grocery shopping at Desi Grocery,-20.00,2. Grocery shopping at Desi Grocery - Groceries,Grocery shopping at Desi Grocery,Groceries
2,2024-01-08,Grocery shopping at Desi Grocery,-4.80,2. Grocery shopping at Desi Grocery - Groceries,Grocery shopping at Desi Grocery,Groceries
3,2024-03-04,Grocery shopping at Desi Grocery,-8.60,2. Grocery shopping at Desi Grocery - Groceries,Grocery shopping at Desi Grocery,Groceries
4,2024-03-11,Grocery shopping at Desi Grocery,-11.60,2. Grocery shopping at Desi Grocery - Groceries,Grocery shopping at Desi Grocery,Groceries
...,...,...,...,...,...,...
75,2024-04-02,Credit Transfer,75.00,26. Credit Transfer - Financial Transactions,Credit Transfer,Financial Transactions
76,2020-04-30,Credit Transfer,1150.00,26. Credit Transfer - Financial Transactions,Credit Transfer,Financial Transactions
77,2024-01-02,Grocery shopping at Lidl,-8.28,1. Grocery shopping at Lidl - **Grocery**,Grocery shopping at Lidl,**Grocery**
78,2024-01-05,Grocery shopping at Lidl,-26.37,1. Grocery shopping at Lidl - **Grocery**,Grocery shopping at Lidl,**Grocery**


In [ ]:
df.to_csv("ardata_categorized116.csv", index=False)